# Data Science Project

## Introduction

Este proyecto tiene como objetivo el poder obtener conocimiento a partir de los datos presentes en un conjunto de datos (dataset, por su traducción en ingles), lo que permitirá poder comprobar una o varias hipotesis propuestas.

Para esto, se define lo siguiente:

- **Conjunto de datos.** Se elegió el conjunto de datos "futbol" que contiene información relacionada de diversos jugadores de distintos clubes de todo el mundo; esto con su información estadistica como jugador y su algunos datos personales.


- **Hipotesis a comprobar.** ¿Existe alguna relación entre la edad, nacionalidad, club, pierna preferida, posición, fecha de unión al club, prestamo, fecha de contrado valido, altura, peso y clausula de liberación para poder determinar el valor y precio de venta de un jugador?

<img src="https://www.freewebheaders.com/wp-content/gallery/football/sports-soccer-stadium-night-scene-web-header.jpg" style="width: 100%"/>

## Preparación para el preprocesamiento

### Importación de librerias

Se importando todas las librerias necesarias para este proyecto, donde las mas importante es **pandas** ya que permite la lectura, manipulación y almacenamiento de los datos.

In [1]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.preprocessing import MinMaxScaler
plt.style.use('ggplot')

### Load dataset

Se lee el conjunto de datos elegido pero filtrando las columnas (variables, en Ciencia de Datos) unicamente necesarias, estas se describen a continuación.

- ID
- Age
- Nationality
- Club
- Value
- Wage
- Preferred Foot
- Position
- Joined
- Loaned From
- Contract Valid Until
- Height
- Weight
- Release Clause

Sin embargo, se renombran las columnas que poseen espacios con los mismos nombres pero sustituyendo los espacios con guiones bajos.

In [2]:
dataframe = (pd.read_csv("futbol.csv")).filter(["ID", "Age", "Nationality","Club","Value", "Wage", "Preferred Foot", "Position", "Joined","Loaned From", "Contract Valid Until", "Height", "Weight", "Release Clause"]).rename(columns={"Preferred Foot":'Preferred_foot', 'Loaned From': 'Loaned_from', 'Contract Valid Until':'Contract_Valid_Until', 'Release Clause': 'Release_clause'})

Posteriormente, se visualiza el dataframe (estructura que posee un conjunto de datos) leido.

In [3]:
dataframe

,ID,Age,Nationality,Club,Value,Wage,Preferred_foot,Position,Joined,Loaned_from,Contract_Valid_Until,Height,Weight,Release_clause
0,158023,31,Argentina,FC Barcelona,€110.5M,€565K,Left,RF,"Jul 1, 2004",NaN,2021,5'7,159lbs,€226.5M
1,20801,33,Portugal,Juventus,€77M,€405K,Right,ST,"Jul 10, 2018",NaN,2022,6'2,183lbs,€127.1M
2,190871,26,Brazil,Paris Saint-Germain,€118.5M,€290K,Right,LW,"Aug 3, 2017",NaN,2022,5'9,150lbs,€228.1M
3,193080,27,Spain,Manchester United,€72M,€260K,Right,GK,"Jul 1, 2011",NaN,2020,6'4,168lbs,€138.6M
4,192985,27,Belgium,Manchester City,€102M,€355K,Right,RCM,"Aug 30, 2015",NaN,2023,5'11,154lbs,€196.4M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18202,238813,19,England,Crewe Alexandra,€60K,€1K,Right,CM,"May 3, 2017",NaN,2019,5'9,134lbs,€143K
18203,243165,19,Sweden,Trelleborgs FF,€60K,€1K,Right,ST,"Mar 19, 2018",NaN,2020,6'3,170lbs,€113K
18204,241638,16,England,Cambridge United,€60K,€1K,Right,ST,"Jul 1, 2017",NaN,2021,5'8,148lbs,€165K
18205,246268,17,England,Tranmere Rovers,€60K,€1K,Right,RW,"Apr 24, 2018",NaN,2019,5'10,154lbs,€143K


## Data mining

### Data cleaning

Debido a que el conjunto de datos elegido no fue elaborado por nosotros se procede a realizar una comprobación de los registros existentes en este, el cual tiene como objetivo de corroborar si existen registros nulos o vacios.
Comprobaremos el tipo de dato que tiene cada columna

In [4]:
pd.DataFrame(dataframe.dtypes, columns=['Tipo de dato'])

,Tipo de dato
ID,int64
Age,int64
Nationality,object
Club,object
Value,object
Wage,object
Preferred_foot,object
Position,object
Joined,object
Loaned_from,object


Para disminuir el tiempo de procesamiento, cambiaremos el tipo de dato int64 a uno de menor cantidad de bits, pero primero debemos comprobar que ID y Age tengan el máximo aceptable por el tipo de dato.

In [5]:
pd.DataFrame(dataframe.max(numeric_only=True), columns=['Tipo de dato'])

,Tipo de dato
ID,246620
Age,45


Primero comprobaremos que no haya ningún problema con este cambio

In [61]:
(dataframe.ID.max() < np.iinfo(np.uint32).max) & (dataframe.Age.max() < np.iinfo(np.uint8).max)

True

Sabiendo que estos datos por naturaleza no deben tener valores negativos, pero nos aseguramos de que así sea

In [62]:
(dataframe.ID.min()>= np.iinfo(np.uint32).min) & (dataframe.Age.min()>= np.iinfo(np.uint8).min)

True

In [7]:
dataframe = dataframe.astype({"ID": np.uint32, 'Age': np.uint8})

Visualizar cantidad de datos nulos por columna

In [8]:
pd.DataFrame(dataframe.isnull().sum(), columns=['Datos nulos'])

,Datos nulos
ID,0
Age,0
Nationality,0
Club,241
Value,0
Wage,0
Preferred_foot,48
Position,60
Joined,1553
Loaned_from,16943


**Nota importante:** teniendo en cuenta el contexto de los datos, se puede mencionar que los datos nulos mas importantes son todos los que no pertenescan a **Release_clause** porque estos pueden categorizarse de otra manera.

Como primer paso, primer visualizamos los registros que se intersectan con valores nulos o vacios de acuerdo a los siguientes datos:

- Preferred_foot
- Position
- Joined
- Loaned_from
- Contract_Valid_Until
- Height
- Weight

In [9]:
dataframe[pd.isnull(dataframe[['Preferred_foot', 'Position', 'Joined', 'Loaned_from', 'Contract_Valid_Until','Height', 'Weight']]).all(axis=1)]

,ID,Age,Nationality,Club,Value,Wage,Preferred_foot,Position,Joined,Loaned_from,Contract_Valid_Until,Height,Weight,Release_clause
13236,177971,33,Scotland,Rochdale,€120K,€1K,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13237,195380,29,Nicaragua,Boyacá Chicó FC,€300K,€1K,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13238,139317,35,England,Notts County,€140K,€3K,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13239,240437,20,Italy,Brescia,€425K,€1K,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13240,209462,24,England,Hamilton Academical FC,€400K,€1K,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13241,219702,21,Poland,Śląsk Wrocław,€425K,€1K,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13242,225590,23,Argentina,Club Atlético Aldosivi,€450K,€2K,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13243,233782,19,England,Everton,€600K,€5K,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13244,239158,30,Denmark,Hobro IK,€230K,€2K,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13245,242998,21,Brazil,HJK Helsinki,€425K,€1K,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Posteriormente, se eliminan todos los registros que se intesectan partiendo de una columna común que en este caso es **Position**. Esto elimina un total de 48 registros del dataframe.

In [10]:
dataframe.dropna(subset=['Position'], inplace=True)

Seguido a esto, se visualiza el dataframe.

In [11]:
dataframe

,ID,Age,Nationality,Club,Value,Wage,Preferred_foot,Position,Joined,Loaned_from,Contract_Valid_Until,Height,Weight,Release_clause
0,158023,31,Argentina,FC Barcelona,€110.5M,€565K,Left,RF,"Jul 1, 2004",NaN,2021,5'7,159lbs,€226.5M
1,20801,33,Portugal,Juventus,€77M,€405K,Right,ST,"Jul 10, 2018",NaN,2022,6'2,183lbs,€127.1M
2,190871,26,Brazil,Paris Saint-Germain,€118.5M,€290K,Right,LW,"Aug 3, 2017",NaN,2022,5'9,150lbs,€228.1M
3,193080,27,Spain,Manchester United,€72M,€260K,Right,GK,"Jul 1, 2011",NaN,2020,6'4,168lbs,€138.6M
4,192985,27,Belgium,Manchester City,€102M,€355K,Right,RCM,"Aug 30, 2015",NaN,2023,5'11,154lbs,€196.4M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18202,238813,19,England,Crewe Alexandra,€60K,€1K,Right,CM,"May 3, 2017",NaN,2019,5'9,134lbs,€143K
18203,243165,19,Sweden,Trelleborgs FF,€60K,€1K,Right,ST,"Mar 19, 2018",NaN,2020,6'3,170lbs,€113K
18204,241638,16,England,Cambridge United,€60K,€1K,Right,ST,"Jul 1, 2017",NaN,2021,5'8,148lbs,€165K
18205,246268,17,England,Tranmere Rovers,€60K,€1K,Right,RW,"Apr 24, 2018",NaN,2019,5'10,154lbs,€143K


Nuevamente, se procede a realizar una comprobación de los registros para corroborar en cuantos aun persisten registros nulos o vacios.

In [12]:
pd.DataFrame(dataframe.isnull().sum(), columns=['Datos nulos'])

,Datos nulos
ID,0
Age,0
Nationality,0
Club,229
Value,0
Wage,0
Preferred_foot,0
Position,0
Joined,1493
Loaned_from,16883


**Nota importante:** se puede observa que ahora existen 229 registros que no poseen un **Club** y **Contract_Valid_Until** por lo cual se puede deducir que hay una relación.

Partiendo de primer paso anterior, se procede a visualizar los registros que se intersectan con valores nulos o vacios de acuerdo a los siguientes datos:

- Joined
- Loaned_from
- Contract_Valid_Until

Buscamos la intersección de las variables con valores nulos así como se hizo el paso anterior

In [13]:
dataframe[pd.isnull(dataframe[['Joined', 'Loaned_from', 'Contract_Valid_Until']]).all(axis=1)]

,ID,Age,Nationality,Club,Value,Wage,Preferred_foot,Position,Joined,Loaned_from,Contract_Valid_Until,Height,Weight,Release_clause
452,207439,24,Argentina,NaN,€0,€0,Right,CM,NaN,NaN,NaN,5'11,165lbs,NaN
538,156713,33,Sweden,NaN,€0,€0,Right,LCB,NaN,NaN,NaN,6'4,185lbs,NaN
568,229909,26,Russia,NaN,€0,€0,Right,GK,NaN,NaN,NaN,6'2,176lbs,NaN
677,187347,29,Russia,NaN,€0,€0,Right,RB,NaN,NaN,NaN,5'10,154lbs,NaN
874,187607,29,Russia,NaN,€0,€0,Right,ST,NaN,NaN,NaN,6'5,201lbs,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16947,225489,23,Iceland,NaN,€0,€0,Right,GK,NaN,NaN,NaN,6'5,187lbs,NaN
16976,236774,21,India,NaN,€0,€0,Left,LW,NaN,NaN,NaN,5'9,154lbs,NaN
17008,234234,27,Bulgaria,NaN,€0,€0,Right,LDM,NaN,NaN,NaN,6'0,163lbs,NaN
17197,245164,21,India,NaN,€0,€0,Right,GK,NaN,NaN,NaN,6'2,176lbs,NaN


Posteriormente, se eliminan todos los registros que se intesectan partiendo de una columna común que en este caso es **Contract_Valid_Until**. Esto elimina un total de 229 registros del dataframe.

In [14]:
dataframe.dropna(subset=['Contract_Valid_Until'], inplace=True)

Seguido a esto, se visualiza el dataframe.

In [15]:
dataframe

,ID,Age,Nationality,Club,Value,Wage,Preferred_foot,Position,Joined,Loaned_from,Contract_Valid_Until,Height,Weight,Release_clause
0,158023,31,Argentina,FC Barcelona,€110.5M,€565K,Left,RF,"Jul 1, 2004",NaN,2021,5'7,159lbs,€226.5M
1,20801,33,Portugal,Juventus,€77M,€405K,Right,ST,"Jul 10, 2018",NaN,2022,6'2,183lbs,€127.1M
2,190871,26,Brazil,Paris Saint-Germain,€118.5M,€290K,Right,LW,"Aug 3, 2017",NaN,2022,5'9,150lbs,€228.1M
3,193080,27,Spain,Manchester United,€72M,€260K,Right,GK,"Jul 1, 2011",NaN,2020,6'4,168lbs,€138.6M
4,192985,27,Belgium,Manchester City,€102M,€355K,Right,RCM,"Aug 30, 2015",NaN,2023,5'11,154lbs,€196.4M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18202,238813,19,England,Crewe Alexandra,€60K,€1K,Right,CM,"May 3, 2017",NaN,2019,5'9,134lbs,€143K
18203,243165,19,Sweden,Trelleborgs FF,€60K,€1K,Right,ST,"Mar 19, 2018",NaN,2020,6'3,170lbs,€113K
18204,241638,16,England,Cambridge United,€60K,€1K,Right,ST,"Jul 1, 2017",NaN,2021,5'8,148lbs,€165K
18205,246268,17,England,Tranmere Rovers,€60K,€1K,Right,RW,"Apr 24, 2018",NaN,2019,5'10,154lbs,€143K


Nuevamente, se procede a realizar una comprobación de los registros para corroborar en cuantos aun persisten registros nulos o vacios.

In [16]:
pd.DataFrame(dataframe.isnull().sum(), columns=['Datos nulos'])

,Datos nulos
ID,0
Age,0
Nationality,0
Club,0
Value,0
Wage,0
Preferred_foot,0
Position,0
Joined,1264
Loaned_from,16654


**Nota importante:** se puede observar que la columna de **Joined** posiblemente sea excluyente con **Loaned From** ya que al hacer la suma aritmetica de sus valores nulos dan como resultado el total del dataframe.

Partiendo de lo observado, se procede a comprobar esta relación de excluyencia realizando la interseción de los registros de acuerdo a los siguientes criterios:

- Joined (con valores no nulos o vacios)
- Loaned_from

In [17]:
dataframe[dataframe.Loaned_from.isnull() & dataframe.Joined.isnull()==False]

,ID,Age,Nationality,Club,Value,Wage,Preferred_foot,Position,Joined,Loaned_from,Contract_Valid_Until,Height,Weight,Release_clause
0,158023,31,Argentina,FC Barcelona,€110.5M,€565K,Left,RF,"Jul 1, 2004",NaN,2021,5'7,159lbs,€226.5M
1,20801,33,Portugal,Juventus,€77M,€405K,Right,ST,"Jul 10, 2018",NaN,2022,6'2,183lbs,€127.1M
2,190871,26,Brazil,Paris Saint-Germain,€118.5M,€290K,Right,LW,"Aug 3, 2017",NaN,2022,5'9,150lbs,€228.1M
3,193080,27,Spain,Manchester United,€72M,€260K,Right,GK,"Jul 1, 2011",NaN,2020,6'4,168lbs,€138.6M
4,192985,27,Belgium,Manchester City,€102M,€355K,Right,RCM,"Aug 30, 2015",NaN,2023,5'11,154lbs,€196.4M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18202,238813,19,England,Crewe Alexandra,€60K,€1K,Right,CM,"May 3, 2017",NaN,2019,5'9,134lbs,€143K
18203,243165,19,Sweden,Trelleborgs FF,€60K,€1K,Right,ST,"Mar 19, 2018",NaN,2020,6'3,170lbs,€113K
18204,241638,16,England,Cambridge United,€60K,€1K,Right,ST,"Jul 1, 2017",NaN,2021,5'8,148lbs,€165K
18205,246268,17,England,Tranmere Rovers,€60K,€1K,Right,RW,"Apr 24, 2018",NaN,2019,5'10,154lbs,€143K


Con la operación anterior se puede comprobar de forma consistente la relación de excluyencia entre estas columnas, sin embargo; se realiza una ultima comprobación para validar si el tamaño de los registros intersectados es el mismo que el tamaño del mismo dataframe.

In [18]:
dataframe[dataframe.Loaned_from.isnull() & dataframe.Joined.isnull()==False].values.shape == dataframe.shape

True

**Nota importante:** partiendo de la comprobación anterior, se puede verificar que el tamaño de los registros intersectados son iguales al tamaño del dataframe.

Una vez comprobado, validado y verificado la relación de excluyencia entre las columnas **Joined** y **Loaned_from**, es necesario realizar las modificaciones a los datos presentes en estes. Sin embargo, se necesita tener en cuenta lo siguiente:

- Las columnas son una cadena de texto con información compleja.
- La columna **Joined** se refiere a la fecha que el jugador se unió al club al que pertenece actualmente.
- La columna **Loaned_from** se refiere al club al que fue prestado incluyendo la fecha.

Las modificaciones propuestas consisten en realizar un renombrado de la columna **Joined** a **Loaned**, que sea de tipo booleana y que indiquen si un jugador está en préstamo (valor true indica que hay información en la columna **Loaned_from** y false indica que hay información en la columna **Joined**).

Como primer paso de las modificaciones propuestas, se renombra la columna **Joined** a **Loaned**.

In [19]:
dataframe.rename({'Joined': 'Loaned'}, axis=1, inplace=True)

Posteriormente, se reemplazan los registros de la columna **Loaned** (anteriormente Joined) partiendo del siguiente criterio:
- Si presenta información entonces el nuevo valor del registro es true.
- Si no presenta información entonces el nuevo valor del registro es false.

In [20]:
dataframe.Loaned = dataframe.Loaned.isnull()

Para comprobar que la operación haya provisto el remplazo esperado, se procede a visualizar la siguiente intersection de los registros de acuerdo a los siguientes datos:
- La columna **Loaned** en true
- La columna **Loaned_from** con registros nulos o vacios.

In [21]:
dataframe[dataframe.Loaned == True].head(n=10)

,ID,Age,Nationality,Club,Value,Wage,Preferred_foot,Position,Loaned,Loaned_from,Contract_Valid_Until,Height,Weight,Release_clause
28,198710,26,Colombia,FC Bayern München,€69.5M,€315K,Left,LAM,True,Real Madrid,"Jun 30, 2019",5'11,172lbs,NaN
38,167664,30,Argentina,Milan,€57M,€245K,Right,LS,True,Juventus,"Jun 30, 2019",6'1,196lbs,NaN
91,187961,29,Brazil,Guangzhou Evergrande Taobao FC,€37M,€235K,Right,LDM,True,FC Barcelona,"Jun 30, 2019",6'0,157lbs,NaN
166,212523,24,Brazil,Guangzhou Evergrande Taobao FC,€36.5M,€18K,Left,CAM,True,SL Benfica,"Dec 31, 2018",6'3,176lbs,NaN
176,207410,24,Croatia,Chelsea,€35M,€165K,Right,LCM,True,Real Madrid,"Jun 30, 2019",5'9,172lbs,NaN
332,216409,24,Italy,Inter,€22.5M,€57K,Left,RM,True,Sassuolo,"Jun 30, 2019",5'7,148lbs,NaN
354,203890,26,Croatia,Inter,€18M,€57K,Right,RB,True,Atlético Madrid,"Jun 30, 2019",5'11,168lbs,NaN
357,202685,27,Italy,Torino,€21M,€48K,Left,RS,True,Valencia CF,"Jun 30, 2019",6'1,185lbs,NaN
427,226226,22,Argentina,Real Betis,€20M,€72K,Left,RCM,True,Paris Saint-Germain,"Jun 30, 2019",5'9,154lbs,NaN
434,219809,23,France,Milan,€17M,€85K,Right,LCM,True,Chelsea,"Jun 30, 2019",6'2,187lbs,NaN


Posteriormente, se visualiza la siguiente intersection de los registros de acuerdo a los siguientes datos:
- La columna **Loaned** en false
- La columna **Loaned_from** con registros nulos o vacios.

In [22]:
dataframe[dataframe.Loaned == False].head(n=10)

,ID,Age,Nationality,Club,Value,Wage,Preferred_foot,Position,Loaned,Loaned_from,Contract_Valid_Until,Height,Weight,Release_clause
0,158023,31,Argentina,FC Barcelona,€110.5M,€565K,Left,RF,False,NaN,2021,5'7,159lbs,€226.5M
1,20801,33,Portugal,Juventus,€77M,€405K,Right,ST,False,NaN,2022,6'2,183lbs,€127.1M
2,190871,26,Brazil,Paris Saint-Germain,€118.5M,€290K,Right,LW,False,NaN,2022,5'9,150lbs,€228.1M
3,193080,27,Spain,Manchester United,€72M,€260K,Right,GK,False,NaN,2020,6'4,168lbs,€138.6M
4,192985,27,Belgium,Manchester City,€102M,€355K,Right,RCM,False,NaN,2023,5'11,154lbs,€196.4M
5,183277,27,Belgium,Chelsea,€93M,€340K,Right,LF,False,NaN,2020,5'8,163lbs,€172.1M
6,177003,32,Croatia,Real Madrid,€67M,€420K,Right,RCM,False,NaN,2020,5'8,146lbs,€137.4M
7,176580,31,Uruguay,FC Barcelona,€80M,€455K,Right,RS,False,NaN,2021,6'0,190lbs,€164M
8,155862,32,Spain,Real Madrid,€51M,€380K,Right,RCB,False,NaN,2020,6'0,181lbs,€104.6M
9,200389,25,Slovenia,Atlético Madrid,€68M,€94K,Right,GK,False,NaN,2021,6'2,192lbs,€144.5M


Partiendo de lo anterior, se puede corroborar de forma consistente que la operación de reemplazamiento de los registros de la columna **Loaned** ha sido lo esperado.

Tomando lo mencionado, se procede a eliminar la columna **Loaned_from** debido a que su información se incluye en la columna **Loaned** de forma consistente.

In [23]:
dataframe.drop(['Loaned_from'], axis=1, inplace=True)

Seguido a esto, se visualiza el dataframe.

In [24]:
dataframe

,ID,Age,Nationality,Club,Value,Wage,Preferred_foot,Position,Loaned,Contract_Valid_Until,Height,Weight,Release_clause
0,158023,31,Argentina,FC Barcelona,€110.5M,€565K,Left,RF,False,2021,5'7,159lbs,€226.5M
1,20801,33,Portugal,Juventus,€77M,€405K,Right,ST,False,2022,6'2,183lbs,€127.1M
2,190871,26,Brazil,Paris Saint-Germain,€118.5M,€290K,Right,LW,False,2022,5'9,150lbs,€228.1M
3,193080,27,Spain,Manchester United,€72M,€260K,Right,GK,False,2020,6'4,168lbs,€138.6M
4,192985,27,Belgium,Manchester City,€102M,€355K,Right,RCM,False,2023,5'11,154lbs,€196.4M
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18202,238813,19,England,Crewe Alexandra,€60K,€1K,Right,CM,False,2019,5'9,134lbs,€143K
18203,243165,19,Sweden,Trelleborgs FF,€60K,€1K,Right,ST,False,2020,6'3,170lbs,€113K
18204,241638,16,England,Cambridge United,€60K,€1K,Right,ST,False,2021,5'8,148lbs,€165K
18205,246268,17,England,Tranmere Rovers,€60K,€1K,Right,RW,False,2019,5'10,154lbs,€143K


Nuevamente, se procede a realizar una comprobación de los registros para corroborar en cuantos aun persisten registros nulos o vacios.

In [25]:
pd.DataFrame(dataframe.isnull().sum(), columns=['Datos nulos'])

,Datos nulos
ID,0
Age,0
Nationality,0
Club,0
Value,0
Wage,0
Preferred_foot,0
Position,0
Loaned,0
Contract_Valid_Until,0


**Nota importante:** se puede observar que solamnente la columna **Release_clause** es la que contiene valores nulos o vacios.

Partiendo del contexto, los valores nulos o vacios existentes en los registros de la columna **Release_clause** puede inferir a que sea una cláusula de recesión de 0 euros, se procede a reemplazar los valores de los registros por una cadena "0".

In [26]:
dataframe.loc[dataframe.Release_clause.isnull(),'Release_clause']='0'

Posteriormente, se comprueba que sean los mismos registros presentes en la columna **Release_clause** a los que se hayan reemplazado su valor nulo con la cadena "0".

In [27]:
dataframe[dataframe.Release_clause =='0']

,ID,Age,Nationality,Club,Value,Wage,Preferred_foot,Position,Loaned,Contract_Valid_Until,Height,Weight,Release_clause
28,198710,26,Colombia,FC Bayern München,€69.5M,€315K,Left,LAM,True,"Jun 30, 2019",5'11,172lbs,0
38,167664,30,Argentina,Milan,€57M,€245K,Right,LS,True,"Jun 30, 2019",6'1,196lbs,0
91,187961,29,Brazil,Guangzhou Evergrande Taobao FC,€37M,€235K,Right,LDM,True,"Jun 30, 2019",6'0,157lbs,0
166,212523,24,Brazil,Guangzhou Evergrande Taobao FC,€36.5M,€18K,Left,CAM,True,"Dec 31, 2018",6'3,176lbs,0
176,207410,24,Croatia,Chelsea,€35M,€165K,Right,LCM,True,"Jun 30, 2019",5'9,172lbs,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17979,241341,21,China PR,Guizhou Hengfeng FC,€60K,€2K,Right,CM,True,"Dec 31, 2018",6'0,172lbs,0
18026,241444,21,China PR,Guizhou Hengfeng FC,€50K,€2K,Right,LM,True,"Dec 31, 2018",5'10,143lbs,0
18031,243506,20,China PR,Stabæk Fotball,€40K,€2K,Right,RB,True,"Dec 31, 2018",5'7,143lbs,0
18056,244872,19,Italy,Ascoli,€60K,€3K,Left,CM,True,"Jun 30, 2019",5'10,143lbs,0


**Nota importante:** se puede observar que es la misma cantidad de los valores nulos o vacios iniciales de la columna **Release_clause** a los que se reemplazo con la cadena "0".

Nuevamente, se procede a realizar una comprobación de los registros para corroborar en cuantos aun persisten registros nulos o vacios.

In [28]:
pd.DataFrame(dataframe.isnull().sum(), columns=['Datos nulos'])

,Datos nulos
ID,0
Age,0
Nationality,0
Club,0
Value,0
Wage,0
Preferred_foot,0
Position,0
Loaned,0
Contract_Valid_Until,0


**Nota importante:** se puede observar que ya no existen todos los valores nulos o vacios en todas las columnas del dataframe.

Una vez modificado los valores nulos o vacios, se procede a se analizar el tipo de dato de todas las columnas del dataframe.

In [29]:
pd.DataFrame(dataframe.dtypes, columns=['Tipo de dato'])

,Tipo de dato
ID,int32
Age,int8
Nationality,object
Club,object
Value,object
Wage,object
Preferred_foot,object
Position,object
Loaned,bool
Contract_Valid_Until,object


**Nota importante:** se puede observar que todas las columnas corresponden a su tipo de dato con la excepción de las columnas **Value**, **Wage**, **Height** y **Release_clause** son tipo cadena (object), cuando realmente estos deberian ser de tipo numérico (int64).

Como primer paso, se procede a convertir las columnas **Value**, **Wage** y **Release_clause** que representan una cantidad de dinero evaluado en euros. Estos contienen de forma adicional lo siguiente:
- Signo de euro (€).
- Posfijo de miles (K) o millones (M).

Seguidamente, se comprueba que todos los registros posean un formato antes mencionado.

In [69]:
all([re.search('€*(\d*.+\d*[MK]*|0)', value) for tupla in dataframe[['Value', 'Wage', 'Release_clause']] for value in tupla])

True

Una vez comprobado, se procede a eliminar los caracteres innecesarios y a convertir de forma inmediata los valores de miles o millones asi como tambien definir el nuevo valor de tipo flotante para los registros existentes de las columnas **Value**, **Wage** y **Release Clause**.

In [ ]:
dataframe[['Value','Wage', 'Release_clause']] = dataframe[['Value','Wage', 'Release_clause']].apply(lambda x: [tupla[1]*1000 if ('M' in tupla[0]) else tupla[1] for tupla in [(value, float(re.findall('(\d+(?:\.\d+)?)', value)[0])) for value in x]])

Como segundo paso, se procede a convertir las columna **Weight** que representan una cantidad de peso evaluado en libras. Estos contienen de forma adicional lo siguiente:
- Posfijo de libra (lbs).

Posteriormente, se comprueba que todos los registros posean un formato antes mencionado.

Validamos que cumpla la siguiente expresión regular

In [50]:
all([re.findall('(\d+(?:\.\d+)?)', value) for value in dataframe.Weight])

True

Una vez validado, comprobaremos cuál es el elemento con la longitud máxima para asignar el tipo de dato óptimo para la variable

In [53]:
max([len(re.findall('(\d+(?:\.\d+)?)', value)[0]) for value in dataframe.Weight])

3

Comprobando que la cadena de texto numérica a convertir es de longitud 3 en el caso máximo, se nos permitiría emplear float16 sin problema

Una vez comprobado, se procede a eliminar los caracteres innecesarios y a definir el nuevo valor de tipo flotante de la columna **Weigth**.

In [ ]:
dataframe.Weight = dataframe.Weight.apply(lambda x: np.float16(re.findall('(\d+(?:\.\d+)?)', x)[0]))

Como tercer paso, se procede a convertir las columna **Height** que representan una cantidad de altura evaluado en pulgadas. Estos contienen de forma adicional lo siguiente:
- Infijo de pulgadas (').

Seguidamente, se comprueba que todos los registros posean un formato antes mencionado.

In [ ]:
all([re.findall('(\d\'\d*)', value) for value in dataframe.Height])

Una vez comprobado, se procede a reemplazar el caracter de pulgada, convertir y definir el nuevo valor de tipo flotante de la columna **Heigth**.

In [ ]:
dataframe.Height = [y[0]*12+y[1] for y in [[int(y) for y in x.split("'")] for x in dataframe.Height]]

**Nota importante:** la conversión aplicada es necesaria para poder interpretar de forma correcta el valor de altura compuesta.

Como cuarto paso, se procede a convertir las columna **Contract_Valid_Until** que representan una fecha pero sin un formato establecido. Estos contienen de forma adicional lo siguiente:
- Mes (3 caracteres).
- Dia (1 a 2 caracteres).
- Año (4 caracteres).

Seguidamente, se comprueba que todos los registros posean por lo mínimo el año del contrato.

In [ ]:
all([re.findall('\d{4}', x)[0] for x in dataframe.Contract_Valid_Until])

Una vez comprobado, se procede a eliminar los caracteres innesesarios y a definir el nuevo valor de tipo entero de la columna **Contract_Valid_Until**.

In [ ]:
dataframe.Contract_Valid_Until = dataframe.Contract_Valid_Until.apply(lambda x: int(re.findall('\d{4}', x)[0]))

Por ultimo, se procede a comprobar los registros de la columna **Preferred_foot**.

In [ ]:
pd.DataFrame((lambda x:{'Preferred_foot':x.index})(dataframe.Preferred_foot.value_counts()))

**Nota importante:** se puede observar que esta columna solamente contiene valores entre las cadenas "Left" o "Righ".

Finalmente, se comprueba el tipo de dato de cada columna del dataframe.

In [ ]:
pd.DataFrame(dataframe.dtypes, columns=['Tipo de dato'])

Seguido a esto, se visualiza el dataframe.

In [ ]:
dataframe

**Nota importante:** se puede observar que las modificaciones han dado como resultado la reducción una columna y el completado de los datos en los valores de registros que antes contenian valores nulos o vacios.

## Normalización de los registros de datos

Partiendo de los datos modificados, se procede a crear una copia del dataframe con el objetivo de modificar los datos para posteriormente normalizarlos, pero sin modificar el dataframe original.

In [ ]:
normalizedDataframe = dataframe.copy()

Se visualiza la copia del dataframe creado.

In [ ]:
normalizedDataframe

**Nota importante:** debido a que existen columnas del dataframe de tipo cadena estos no se pueden normalizar, por lo que se requiere crear subconjunto de datos para relacionar estos valores para que posteriormente sean normalizados.

Como primer paso, se crea un subconjunto de datos para representar la frecuencia de los valores duplicados para la columna **Nationality**.

In [ ]:
countryDataframe = pd.DataFrame((lambda x:{'Nationality':x.index, 'Frequency':x.values})(dataframe.Nationality.value_counts()))

Se visualiza el nuevo subconjunto creado.

In [ ]:
countryDataframe

Como segundo paso, se crea un subconjunto de datos para representar la frecuencia de los valores duplicados para la columna **Club**.

In [ ]:
clubDataframe = pd.DataFrame((lambda x:{'Club':x.index, 'Frequency':x.values})(dataframe.Club.value_counts()))

Se visualiza el nuevo subconjunto creado.

In [ ]:
clubDataframe

Como tercer paso, se crea un subconjunto de datos para representar la frecuencia de los valores duplicados para la columna **Preferred_foot**.

In [ ]:
preferredFootDataframe = pd.DataFrame((lambda x:{'Preferred_foot':x.index, 'Frequency':x.values})(dataframe.Preferred_foot.value_counts()))

Se visualiza el nuevo subconjunto creado.

In [ ]:
preferredFootDataframe

Como cuarto paso, se crea un subconjunto de datos para representar la frecuencia de los valores duplicados para la columna **Position**.

In [ ]:
positionDataframe = pd.DataFrame((lambda x:{'Position':x.index, 'Frequency':x.values})(dataframe.Position.value_counts()))

Se visualiza el nuevo subconjunto creado.

In [ ]:
positionDataframe

Como quinto paso, se crea un subconjunto de datos para representar la frecuencia de los valores duplicados para la columna **Loaned**.

In [ ]:
loanedDataframe = pd.DataFrame((lambda x:{'Loaned':x.index, 'Frequency':x.values})(dataframe.Loaned.value_counts()))

Se visualiza el nuevo subconjunto creado.

In [ ]:
loanedDataframe

Como ultimo paso, se indexa las columnas **Nationality**, **Club**, **Preferred_foot**, **Position** y **Loaned** con los subconjuntos de datos creados en los pasos anteriores, esto sustituyendo los valores del nuevo dataframe con los nuevos pertenecienctes en los subconjuntos.

In [ ]:
for tupla in {'Nationality': countryDataframe, 'Club':clubDataframe, 'Preferred_foot':preferredFootDataframe, 'Position':positionDataframe, 'Loaned':loanedDataframe}.items(): normalizedDataframe[tupla[0]] = dataframe[tupla[0]].apply(lambda x:(tupla[1][tupla[0]].loc[tupla[1][tupla[0]] == x].index[0]))

Finalmente, se procede a normalizar todos los registros de todas las columnas del nuevo dataframe.

In [ ]:
normalizedDataframe = pd.DataFrame(MinMaxScaler().fit_transform(normalizedDataframe), columns=normalizedDataframe.columns)

Se procede a visualizar el nuevo dataframe con los registros normalizados.

In [ ]:
normalizedDataframe

## Visualización de los registros de datos

Partiendo del dataframe normalizados, se procede a generar gráficas con el objetivo de observar de forma visual el comportamiento de los datos.

Como primer paso, se visualiza las graficas de tipo histograma de todas las columnas del dataframe normalizado.

In [ ]:
normalizedDataframe.hist(figsize=(30,30))
plt.show()

Como segundo paso, se visualiza las gráficas de tipo dispersión todas las columnas a excepción de la columna **ID** común.

Se procede a visualizar gráfica de tipo dispersión entre la columna **ID** y **Age**.

In [ ]:
normalizedDataframe.plot.scatter(x = 'ID', y = 'Age', s = 10, c = 'red', figsize=(5,5));
plt.show()

Se procede a visualizar gráfica de tipo dispersión entre la columna **ID** y **Nationality**.

In [ ]:
normalizedDataframe.plot.scatter(x = 'ID', y = 'Nationality', s = 10, c = 'red', figsize=(5,5));
plt.show()

Se procede a visualizar gráfica de tipo dispersión entre la columna **ID** y **Club**.

In [ ]:
normalizedDataframe.plot.scatter(x = 'ID', y = 'Club', s = 10, c = 'red', figsize=(5,5));
plt.show()

Se procede a visualizar gráfica de tipo dispersión entre la columna **ID** y **Value**.

In [ ]:
normalizedDataframe.plot.scatter(x = 'ID', y = 'Value', s = 10, c = 'red', figsize=(5,5));
plt.show()

Se procede a visualizar gráfica de tipo dispersión entre la columna **ID** y **Wage**.

In [ ]:
normalizedDataframe.plot.scatter(x = 'ID', y = 'Wage', s = 10, c = 'red', figsize=(5,5));
plt.show()

Se procede a visualizar gráfica de tipo dispersión entre la columna **ID** y **Preferred_foot**.

In [ ]:
normalizedDataframe.plot.scatter(x = 'ID', y = 'Preferred_foot', s = 10, c = 'red', figsize=(5,5));
plt.show()

Se procede a visualizar gráfica de tipo dispersión entre la columna **ID** y **Position**.

In [ ]:
normalizedDataframe.plot.scatter(x = 'ID', y = 'Position', s = 10, c = 'red', figsize=(5,5));
plt.show()

Se procede a visualizar gráfica de tipo dispersión entre la columna **ID** y **Loaned**.

In [ ]:
normalizedDataframe.plot.scatter(x = 'ID', y = 'Loaned', s = 10, c = 'red', figsize=(5,5));
plt.show()

Se procede a visualizar gráfica de tipo dispersión entre la columna **ID** y **Contract_Valid_Util**.

In [ ]:
normalizedDataframe.plot.scatter(x = 'ID', y = 'Contract_Valid_Until', s = 10, c = 'red', figsize=(5,5));
plt.show()

Se procede a visualizar gráfica de tipo dispersión entre la columna **ID** y **Heigth**.

In [ ]:
normalizedDataframe.plot.scatter(x = 'ID', y = 'Height', s = 10, c = 'red', figsize=(5,5));
plt.show()

Se procede a visualizar gráfica de tipo dispersión entre la columna **ID** y **Weight**.

In [ ]:
normalizedDataframe.plot.scatter(x = 'ID', y = 'Weight', s = 10, c = 'red', figsize=(5,5));
plt.show()

Se procede a visualizar gráfica de tipo dispersión entre la columna **ID** y **Release_clause**.

In [ ]:
normalizedDataframe.plot.scatter(x = 'ID', y = 'Release_clause', s = 10, c = 'red', figsize=(5,5));
plt.show()

Como último paso, se visualiza la gráfica de tipo correlación para observar las relaciones mas significativas entre las columnas del dataframe normalizado.

In [ ]:
plt.figure(figsize=(30,30))
sb.heatmap(normalizedDataframe.corr(), cmap ='Blues', linewidths = 0.30, annot = True)
plt.show()

In [ ]:
pd.DataFrame(dataframe.isnull().sum(), columns=['Datos nulos'])

**Nota importante:** se puede observar que ya no existen todos los valores nulos o vacios en todas las columnas del dataframe.

Una vez modificado los valores nulos o vacios, se procede a se analizar el tipo de dato de todas las columnas del dataframe.

In [ ]:
pd.DataFrame(dataframe.dtypes, columns=['Tipo de dato'])

**Nota importante:** se puede observar que todas las columnas corresponden a su tipo de dato con la excepción de las columnas **Value**, **Wage**, **Height** y **Release_clause** son tipo cadena (object), cuando realmente estos deberian ser de tipo numérico (int64).

Como primer paso, se procede a convertir las columnas **Value**, **Wage** y **Release_clause** que representan una cantidad de dinero evaluado en euros. Estos contienen de forma adicional lo siguiente:
- Signo de euro (€).
- Posfijo de miles (K) o millones (M).

Seguidamente, se comprueba que todos los registros posean un formato antes mencionado.

In [ ]:
all([re.search('€*(\d*.+\d*[MK]*|0)', value) for tupla in dataframe[['Value', 'Wage', 'Release_clause']] for value in tupla])

Una vez comprobado, se procede a eliminar los caracteres innecesarios y a convertir de forma inmediata los valores de miles o millones asi como tambien definir el nuevo valor de tipo flotante para los registros existentes de las columnas **Value**, **Wage** y **Release Clause**.

In [ ]:
dataframe[['Value','Wage', 'Release_clause']] = dataframe[['Value','Wage', 'Release_clause']].apply(lambda x: [tupla[1]*1000 if ('M' in tupla[0]) else tupla[1] for tupla in [(value, float(re.findall('(\d+(?:\.\d+)?)', value)[0])) for value in x]])

Como segundo paso, se procede a convertir las columna **Weight** que representan una cantidad de peso evaluado en libras. Estos contienen de forma adicional lo siguiente:
- Posfijo de libra (lbs).

Posteriormente, se comprueba que todos los registros posean un formato antes mencionado.

In [ ]:
all([re.findall('(\d+(?:\.\d+)?)lbs', value) for value in dataframe.Weight.values])

Una vez comprobado, se procede a eliminar los caracteres innecesarios y a definir el nuevo valor de tipo flotante de la columna **Weigth**.

In [ ]:
dataframe.Weight = dataframe.Weight.apply(lambda x: float(re.findall('(\d+(?:\.\d+)?)', x)[0]))

Como tercer paso, se procede a convertir las columna **Height** que representan una cantidad de altura evaluado en pulgadas. Estos contienen de forma adicional lo siguiente:
- Infijo de pulgadas (').

Seguidamente, se comprueba que todos los registros posean un formato antes mencionado.

In [ ]:
all([re.findall('(\d\'\d*)', value) for value in dataframe.Height])

Una vez comprobado, se procede a reemplazar el caracter de pulgada y a definir el nuevo valor de tipo flotante de la columna **Heigth**.

In [ ]:
dataframe.Height = [y[0]+(y[1] / 10) for y in [[int(y) for y in x.split("'")] for x in dataframe.Height]]

Como cuarto paso, se procede a convertir las columna **Contract_Valid_Until** que representan una fecha pero sin un formato establecido. Estos contienen de forma adicional lo siguiente:
- Mes (3 caracteres).
- Dia (1 a 2 caracteres).
- Año (4 caracteres).

Seguidamente, se comprueba que todos los registros posean por lo mínimo el año del contrato.

In [ ]:
all([re.findall('\d{4}', x)[0] for x in dataframe.Contract_Valid_Until])

Una vez comprobado, se procede a eliminar los caracteres innesesarios y a definir el nuevo valor de tipo entero de la columna **Contract_Valid_Until**.

In [ ]:
dataframe.Contract_Valid_Until = dataframe.Contract_Valid_Until.apply(lambda x: int(re.findall('\d{4}', x)[0]))

Por ultimo, se procede a comprobar los registros de la columna **Preferred_foot**.

In [ ]:
pd.DataFrame((lambda x:{'Preferred_foot':x.index})(dataframe.Preferred_foot.value_counts()))

**Nota importante:** se puede observar que esta columna solamente contiene valores entre las cadenas "Left" o "Righ".

Finalmente, se comprueba el tipo de dato de cada columna del dataframe.

In [ ]:
pd.DataFrame(dataframe.dtypes, columns=['Tipo de dato'])

Seguido a esto, se visualiza el dataframe.

In [ ]:
dataframe

**Nota importante:** se puede observar que las modificaciones han dado como resultado la reducción una columna y el completado de los datos en los valores de registros que antes contenian valores nulos o vacios.

## Normalización de los registros de datos

Partiendo de los datos modificados, se procede a crear una copia del dataframe con el objetivo de modificar los datos para posteriormente normalizarlos, pero sin modificar el dataframe original.

In [ ]:
normalizedDataframe = dataframe.copy()

Se visualiza la copia del dataframe creado.

In [ ]:
normalizedDataframe

**Nota importante:** debido a que existen columnas del dataframe de tipo cadena estos no se pueden normalizar, por lo que se requiere crear subconjunto de datos para relacionar estos valores para que posteriormente sean normalizados.

Como primer paso, se crea un subconjunto de datos para representar la frecuencia de los valores duplicados para la columna **Nationality**.

In [ ]:
countryDataframe = pd.DataFrame((lambda x:{'Nationality':x.index, 'Frequency':x.values})(dataframe.Nationality.value_counts()))

Se visualiza el nuevo subconjunto creado.

In [ ]:
countryDataframe

Como segundo paso, se crea un subconjunto de datos para representar la frecuencia de los valores duplicados para la columna **Club**.

In [ ]:
clubDataframe = pd.DataFrame((lambda x:{'Club':x.index, 'Frequency':x.values})(dataframe.Club.value_counts()))

Se visualiza el nuevo subconjunto creado.

In [ ]:
clubDataframe

Como tercer paso, se crea un subconjunto de datos para representar la frecuencia de los valores duplicados para la columna **Preferred_foot**.

In [ ]:
preferredFootDataframe = pd.DataFrame((lambda x:{'Preferred_foot':x.index, 'Frequency':x.values})(dataframe.Preferred_foot.value_counts()))

Se visualiza el nuevo subconjunto creado.

In [ ]:
preferredFootDataframe

Como cuarto paso, se crea un subconjunto de datos para representar la frecuencia de los valores duplicados para la columna **Position**.

In [ ]:
positionDataframe = pd.DataFrame((lambda x:{'Position':x.index, 'Frequency':x.values})(dataframe.Position.value_counts()))

Se visualiza el nuevo subconjunto creado.

In [ ]:
positionDataframe

Como quinto paso, se crea un subconjunto de datos para representar la frecuencia de los valores duplicados para la columna **Loaned**.

In [ ]:
loanedDataframe = pd.DataFrame((lambda x:{'Loaned':x.index, 'Frequency':x.values})(dataframe.Loaned.value_counts()))

Se visualiza el nuevo subconjunto creado.

In [ ]:
loanedDataframe

Como ultimo paso, se indexa las columnas **Nationality**, **Club**, **Preferred_foot**, **Position** y **Loaned** con los subconjuntos de datos creados en los pasos anteriores, esto sustituyendo los valores del nuevo dataframe con los nuevos pertenecienctes en los subconjuntos.

In [ ]:
for tupla in {'Nationality': countryDataframe, 'Club':clubDataframe, 'Preferred_foot':preferredFootDataframe, 'Position':positionDataframe, 'Loaned':loanedDataframe}.items(): normalizedDataframe[tupla[0]] = dataframe[tupla[0]].apply(lambda x:(tupla[1][tupla[0]].loc[tupla[1][tupla[0]] == x].index[0]))

Finalmente, se procede a normalizar todos los registros de todas las columnas del nuevo dataframe.

In [ ]:
normalizedDataframe = pd.DataFrame(MinMaxScaler().fit_transform(normalizedDataframe), columns=normalizedDataframe.columns)

Se procede a visualizar el nuevo dataframe con los registros normalizados.

In [ ]:
normalizedDataframe

## Visualización de los registros de datos

Partiendo del dataframe normalizados, se procede a generar gráficas con el objetivo de observar de forma visual el comportamiento de los datos.

Como primer paso, se visualiza las graficas de tipo histograma de todas las columnas del dataframe normalizado.

In [ ]:
normalizedDataframe.hist(figsize=(30,30))
plt.show()

Como segundo paso, se visualiza las gráficas de tipo dispersión todas las columnas a excepción de la columna **ID** común.

Se procede a visualizar gráfica de tipo dispersión entre la columna **ID** y **Age**.

In [ ]:
normalizedDataframe.plot.scatter(x = 'ID', y = 'Age', s = 10, c = 'red', figsize=(5,5));
plt.show()

Se procede a visualizar gráfica de tipo dispersión entre la columna **ID** y **Nationality**.

In [ ]:
normalizedDataframe.plot.scatter(x = 'ID', y = 'Nationality', s = 10, c = 'red', figsize=(5,5));
plt.show()

Se procede a visualizar gráfica de tipo dispersión entre la columna **ID** y **Club**.

In [ ]:
normalizedDataframe.plot.scatter(x = 'ID', y = 'Club', s = 10, c = 'red', figsize=(5,5));
plt.show()

Se procede a visualizar gráfica de tipo dispersión entre la columna **ID** y **Value**.

In [ ]:
normalizedDataframe.plot.scatter(x = 'ID', y = 'Value', s = 10, c = 'red', figsize=(5,5));
plt.show()

Se procede a visualizar gráfica de tipo dispersión entre la columna **ID** y **Wage**.

In [ ]:
normalizedDataframe.plot.scatter(x = 'ID', y = 'Wage', s = 10, c = 'red', figsize=(5,5));
plt.show()

Se procede a visualizar gráfica de tipo dispersión entre la columna **ID** y **Preferred_foot**.

In [ ]:
normalizedDataframe.plot.scatter(x = 'ID', y = 'Preferred_foot', s = 10, c = 'red', figsize=(5,5));
plt.show()

Se procede a visualizar gráfica de tipo dispersión entre la columna **ID** y **Position**.

In [ ]:
normalizedDataframe.plot.scatter(x = 'ID', y = 'Position', s = 10, c = 'red', figsize=(5,5));
plt.show()

Se procede a visualizar gráfica de tipo dispersión entre la columna **ID** y **Loaned**.

In [ ]:
normalizedDataframe.plot.scatter(x = 'ID', y = 'Loaned', s = 10, c = 'red', figsize=(5,5));
plt.show()

Se procede a visualizar gráfica de tipo dispersión entre la columna **ID** y **Contract_Valid_Util**.

In [ ]:
normalizedDataframe.plot.scatter(x = 'ID', y = 'Contract_Valid_Until', s = 10, c = 'red', figsize=(5,5));
plt.show()

Se procede a visualizar gráfica de tipo dispersión entre la columna **ID** y **Heigth**.

In [ ]:
normalizedDataframe.plot.scatter(x = 'ID', y = 'Height', s = 10, c = 'red', figsize=(5,5));
plt.show()

Se procede a visualizar gráfica de tipo dispersión entre la columna **ID** y **Weight**.

In [ ]:
normalizedDataframe.plot.scatter(x = 'ID', y = 'Weight', s = 10, c = 'red', figsize=(5,5));
plt.show()

Se procede a visualizar gráfica de tipo dispersión entre la columna **ID** y **Release_clause**.

In [ ]:
normalizedDataframe.plot.scatter(x = 'ID', y = 'Release_clause', s = 10, c = 'red', figsize=(5,5));
plt.show()

Como último paso, se visualiza la gráfica de tipo correlación para observar las relaciones mas significativas entre las columnas del dataframe normalizado.

In [ ]:
plt.figure(figsize=(30,30))
sb.heatmap(normalizedDataframe.corr(), cmap ='Blues', linewidths = 0.30, annot = True)
plt.show()